### Data exploratory analysis

In [252]:
import pandas as pd
import numpy as np

In [253]:
url = r'C:\Users\dell\Desktop\Project\Berlin_bike_theft_prediction.csv'

df = pd.read_csv(url, sep=',', encoding="utf-8")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31058 entries, 0 to 31057
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Record_date              31058 non-null  object
 1   Record_day               31058 non-null  int64 
 2   Record_month             31058 non-null  int64 
 3   Record_year              31058 non-null  int64 
 4   Stealing_date            31058 non-null  object
 5   Stealing_hour            31058 non-null  object
 6   Report_stealing_date     31058 non-null  object
 7   Report_stealing_hour     31058 non-null  object
 8   Berlin_code_area         31058 non-null  int64 
 9   Bike_value               31058 non-null  int64 
 10  Unsuccesful_attempt      31058 non-null  object
 11  Bike_type                31058 non-null  object
 12  Crime_type               31058 non-null  object
 13  Crime_reason             31058 non-null  object
 14  Administrative_district  31058 non-nul

In [249]:
df.groupby('Record_day_of_week')[['Record_date']].count().sort_values(by='Record_date', ascending=False)

KeyError: 'Record_day_of_week'

In [ ]:
df.groupby('Record_day_of_week')[['Record_date']].count().sort_values(by='Record_date', ascending=True).plot(kind='barh',
                                                                                                                   figsize=(12,10),
                                                                                                                   fontsize=12,
                                                                                                                   title='Amount of crimes by Day of Week',
                                                                                                                   xlabel='Day of week',
                                                                                                                   grid=True)

In [ ]:
df.groupby('Administrative_district')[['Record_date']].count().sort_values(by=['Record_date'], ascending=False)

In [ ]:
df.groupby('Administrative_district')[['Record_date']].count().sort_values(by=['Record_date'], ascending=True).plot(kind='barh',
                                                                                                                          figsize=(12,10),
                                                                                                                          grid=True)

In [ ]:
df.groupby('Bike_type')[['Record_date']].count().sort_values(by='Record_date', ascending=False)

In [ ]:
df.groupby('Crime_type')[['Record_date']].count().sort_values(by='Record_date', ascending=False)

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score, make_scorer

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df.drop(['Record_date'], axis=1, inplace=True)

In [254]:
# Split data to 70% train and 30% test (the size of sample is big almost 31 thousand)
X = df.drop('Crime_type', axis=1)
y = df['Crime_type']

In [255]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                    random_state = 0, stratify = y)

In [256]:
from sklearn.preprocessing import LabelEncoder

In [257]:
X

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_reason,Administrative_district,District,Quarter,Street_name
0,2022-07-20,20,7,2022,2022-07-19,16:00:00,2022-07-19,17:00:00,1100206,600,No,Bike,Aggravated theft,Mitte,Zentrum,Regierungsviertel,Unter den Linden
1,2022-07-20,20,7,2022,2022-07-19,19:00:00,2022-07-20,06:00:00,3500936,869,No,Men,Aggravated theft,Pankow,Südliches Weißensee,Weißensee,Komponistenviertel Weißensee
2,2022-07-20,20,7,2022,2022-07-19,17:00:00,2022-07-19,22:00:00,12100205,1269,No,Men,Aggravated theft,Reinickendorf,Reinickendorf Ost,Ost 2 - Alt-Reinickendorf,Teichstraße
3,2022-07-20,20,7,2022,2022-07-18,20:00:00,2022-07-19,07:00:00,7501030,1000,No,Women,Theft,Tempelhof-Schöneberg,Mariendorf,Mariendorf Nord,Machonstraße
4,2022-07-20,20,7,2022,2022-07-17,10:00:00,2022-07-18,10:00:00,2400624,880,No,Men,Aggravated theft,Friedrichshain-Kreuzberg,Friedrichshain West,Karl-Marx-Allee Süd,Weberwiese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31053,2021-01-02,2,1,2021,2021-01-01,01:00:00,2021-01-01,03:00:00,2100105,2000,No,Mountain,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Nord,Südliche Friedrichstadt,Prinzenstraße
31054,2021-01-01,1,1,2021,2021-01-01,17:00:00,2021-01-01,18:00:00,8401241,888,No,Bike,Theft,Neukölln,Buckow Nord/Rudow,Rudow,Blumenviertel
31055,2021-01-01,1,1,2021,2021-01-01,05:00:00,2021-01-01,08:00:00,10400941,2527,No,Bike,Aggravated theft from basement/floor,Marzahn-Hellersdorf,Kaulsdorf/Mahlsdorf,Mahlsdorf,Pilgramer Straße
31056,2021-01-01,1,1,2021,2021-01-01,20:00:00,2021-01-01,20:00:00,2100104,2800,No,Men,Aggravated theft,Friedrichshain-Kreuzberg,Kreuzberg Nord,Südliche Friedrichstadt,Moritzplatz


In [236]:
ohc = preprocessing.OneHotEncoder()
X = ohc.fit_transform(X)

In [259]:
X

,Record_date,Record_day,Record_month,Record_year,Stealing_date,Stealing_hour,Report_stealing_date,Report_stealing_hour,Berlin_code_area,Bike_value,Unsuccesful_attempt,Bike_type,Crime_reason,Administrative_district,District,Quarter,Street_name
0,565,19,6,1,564,16,564,17,5,503,0,0,0,4,57,103,477
1,565,19,6,1,564,19,565,6,120,771,0,3,0,6,42,131,275
2,565,19,6,1,564,17,564,22,508,1159,0,3,0,7,35,90,456
3,565,19,6,1,563,20,564,7,315,899,0,6,2,10,26,68,313
4,565,19,6,1,562,10,563,10,72,782,0,3,0,1,9,53,496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31053,1,1,0,0,0,1,0,3,53,1712,0,4,0,1,19,121,381
31054,0,0,0,0,0,17,0,18,373,790,0,0,2,5,3,107,65
31055,0,0,0,0,0,5,0,8,462,1996,0,0,1,3,18,67,375
31056,0,0,0,0,0,20,0,20,52,2109,0,3,0,1,19,121,337


In [238]:
print(X)

  (0, 565)	1.0
  (0, 585)	1.0
  (0, 603)	1.0
  (0, 610)	1.0
  (0, 1175)	1.0
  (0, 1193)	1.0
  (0, 1765)	1.0
  (0, 1784)	1.0
  (0, 1796)	1.0
  (0, 2834)	1.0
  (0, 5121)	1.0
  (0, 5122)	1.0
  (0, 5129)	1.0
  (0, 5137)	1.0
  (0, 5202)	1.0
  (0, 5306)	1.0
  (0, 5822)	1.0
  (1, 565)	1.0
  (1, 585)	1.0
  (1, 603)	1.0
  (1, 610)	1.0
  (1, 1175)	1.0
  (1, 1196)	1.0
  (1, 1766)	1.0
  (1, 1773)	1.0
  :	:
  (31056, 4440)	1.0
  (31056, 5121)	1.0
  (31056, 5125)	1.0
  (31056, 5129)	1.0
  (31056, 5134)	1.0
  (31056, 5164)	1.0
  (31056, 5324)	1.0
  (31056, 5682)	1.0
  (31057, 0)	1.0
  (31057, 566)	1.0
  (31057, 597)	1.0
  (31057, 609)	1.0
  (31057, 611)	1.0
  (31057, 1186)	1.0
  (31057, 1201)	1.0
  (31057, 1778)	1.0
  (31057, 1941)	1.0
  (31057, 2524)	1.0
  (31057, 5121)	1.0
  (31057, 5128)	1.0
  (31057, 5129)	1.0
  (31057, 5133)	1.0
  (31057, 5150)	1.0
  (31057, 5249)	1.0
  (31057, 5623)	1.0


In [262]:
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(y)

In [263]:
y

array([1, 1, 1, ..., 0, 1, 1])

In [264]:
# create a pipeline
clf_log_reg = LogisticRegression(random_state = 0)

# create a paramter grid
maxent_grid = {'clf_log_reg__C' : 1, 
                   'clf_log_reg__penalty' : ['l1','l2']}

# check the performance of out of the box logistic regression

#clf_log_reg_baseline = clf_log_reg(LogisticRegression(random_state = 0))

clf_log_reg.fit(X_train, y_train)

clf_log_reg.score(X_test, y_test)

ValueError: could not convert string to float: '2022-02-05'